In [1]:
import finnhub
import yfinance as yf
from pprint import pprint
from datetime import timezone
import numpy as np
import pandas as pd
from datetime import date
import io
from fastai.tabular.core import add_datepart
import csv
pd.options.mode.chained_assignment = None

In [2]:
key = "cb6b5j2ad3i70tu634o0"
finnhub_client = finnhub.Client(api_key=key)

In [3]:
#function implementation

##historical data for stock prices
def gethistoricaldata(ticker, period = "max", interval = "1d"):
    return ticker.history(period = period, interval = interval)
##analyst recommendations daily
def getrecommendations(ticker):
    return ticker.recommendations
##social sentiment
def getsocialsentiment(ticker):
    return finnhub_client.stock_social_sentiment(ticker)

In [4]:
tickers = pd.read_csv('tickers.csv')
tickers = tickers.values.tolist()

In [5]:
def function_getting_data(ticker):
    today_date = date.today()
    # Recode Original Dataframe
    df = gethistoricaldata(ticker, period = "max", interval = "1d")
    df['difference'] = df['Close']-df['Open']
    conditions = [
      (df['difference'] < 0),
      (df['difference'] == 0),
      (df['difference'] > 0)
      ]
    values = [-1, 0, 1]
    df['change'] = np.select(conditions, values)
    df['date_df'] = df.index.format()
    df['today_date_col'] = pd.to_datetime(today_date)
    df['dates_new_format'] = pd.to_datetime(df['date_df'])
    df['date_difference'] = df['dates_new_format'] - df['today_date_col']
    # Recode Recommendation
    df_rec = getrecommendations(ticker)
    df_rec['string_date'] =  df_rec.index.format()
    df_rec['short_date'] = df_rec['string_date'].str.slice(0,10)
    df_rec['today_date_col'] = pd.to_datetime(today_date)
    df_rec['dates_new_format'] = pd.to_datetime(df_rec['short_date'])
    df_rec['To Grade'] = df_rec['To Grade'].str.lower()
    list_to_grade = ['buy','neutral','overweight','outperform','hold','equal-weight','market perform','underperform','sell','strong buy']
    df_rec['To_true_Grade'] = df_rec['To Grade'].isin(list_to_grade)
    df_rec = df_rec[df_rec['To_true_Grade'] == True]
    new_conditions = [
      (df_rec['To Grade'] == "buy"),
      (df_rec['To Grade'] == "neutral"),
      (df_rec['To Grade'] == "overweight"),
      (df_rec['To Grade'] == "hold"),
      (df_rec['To Grade'] == "equal-weight"),
      (df_rec['To Grade'] == "market perform"),
      (df_rec['To Grade'] == "underperform"),
      (df_rec['To Grade'] == "sell"),
      (df_rec['To Grade'] == "strong buy"),
      ]
    new_values = [2, 0, 1, 0, 0, 0, -1, -2, 2]
    df_rec['numeric_grade'] = np.select(new_conditions, new_values)
    df_rec['date_difference'] = (df_rec['dates_new_format'] - df_rec['today_date_col']).dt.days
    # cipd = change in past day (this is something we would like to predict), ciptd = change in past three days, cipsd = change in past seven days, cipm = change in past thirty days
    cipd = df['change'].iloc[-1:][0]
    ciptd = df['change'].iloc[-4:].sum() - cipd
    cipsd = df['change'].iloc[-8:].sum() - cipd
    cipm = df['change'].iloc[-31:].sum() - cipd
    # Difference change - dcpd = difference change in past day (this is something we would like to predict), dcptd = difference change in past three days, dcpsd = difference change in past seven days, dcpm = difference change in past month
    dcpd = df['difference'].iloc[-1:][0]
    dcptd = df['difference'].iloc[-4:].sum() - dcpd
    dcpsd = df['difference'].iloc[-8:].sum() - dcpd
    dcpm = df['difference'].iloc[-31:].sum() - dcpd
    # recommendation level for last seven days, rllsd = 0, if no rec, and = other numerical value if rec; recommendation level for last thirty days, rlltd = 0, if no rec, and = other numerical value if rec
    rllsd = df_rec.loc[df_rec['date_difference'] >= -7, 'numeric_grade'].sum()
    rlltd = df_rec.loc[df_rec['date_difference'] >= -30, 'numeric_grade'].sum()
    # sentiment analysis, sfr = score_from_reddit, sft = score_from_twitter
    sfr = 0
    sft = 0
    result_social_sentiment = getsocialsentiment('AAPL')
    for i in range(len(result_social_sentiment['reddit'])):
        sfr += result_social_sentiment['reddit'][i]['score']
    for i in range(len(result_social_sentiment['twitter'])):
        sft += result_social_sentiment['twitter'][i]['score']
    return cipd, ciptd, cipsd, cipm, dcpd, dcptd, dcpsd, dcpm, rllsd, rlltd, sfr, sft

In [9]:
list_of_lists = []
# big data: 500
# small data: 20

for i in range(20):
    speci_ticker = tickers[i][0]
    try:
      cipd, ciptd, cipsd, cipm, dcpd, dcptd, dcpsd, dcpm, rllsd, rlltd, sfr, sft = function_getting_data(yf.Ticker(speci_ticker))
      list_temp = [speci_ticker, cipd, ciptd, cipsd, cipm, dcpd, dcptd, dcpsd, dcpm, rllsd, rlltd, sfr, sft]
      list_of_lists.append(list_temp)
    except Exception as e:
      print(e)
    print(len(tickers)-i)
df_used_for_ml = pd.DataFrame(list_of_lists,columns=['name_of_stock','cipd(dependent)', 'ciptd', 'cipsd', 'cipm', 'dcpd(dependent)', 'dcptd', 'dcpsd', 'dcpm', 'rllsd', 'rlltd', 'sfr','sft'])

19805
19804
- AACQ: No data found, symbol may be delisted
'NoneType' object has no attribute 'index'
19803
- AACQU: No data found, symbol may be delisted
'NoneType' object has no attribute 'index'
19802
- AACQW: No data found, symbol may be delisted
'NoneType' object has no attribute 'index'
19801
19800
19799
'NoneType' object has no attribute 'index'
19798
'NoneType' object has no attribute 'index'
19797
19796
19795
19794
19793
19792
19791
'NoneType' object has no attribute 'index'
19790
19789
19788
19787
19786


In [10]:
df_used_for_ml.head()

,name_of_stock,cipd(dependent),ciptd,cipsd,cipm,dcpd(dependent),dcptd,dcpsd,dcpm,rllsd,rlltd,sfr,sft
0,AA,-1,1,1,-6,-0.250000,-0.179996,1.84,-10.054993,0,2,-34.470912,-8.154785
1,AACG,-1,3,2,3,-0.050000,0.350000,0.27,0.180000,0,0,-34.470912,-8.154785
2,AAIC,-1,3,4,4,-0.030000,0.190000,0.20,-0.210000,0,0,-34.470912,-8.154785
3,AAL,-1,1,1,0,-0.429999,-0.260000,-0.23,-0.019998,0,0,-34.470912,-8.154785
4,AAN,-1,1,1,-4,-0.369999,0.690001,1.06,-0.259001,0,0,-34.470912,-8.154785


In [11]:
#df_used_for_ml.to_csv('data_large.csv')
df_used_for_ml.to_csv('data_small.csv')

In [ ]:
#df = pd.read_csv("data_large.csv")
df = pd.read_csv("data_small.csv")
newlist = df["name_of_stock"].values.tolist()
for i in range(len(newlist)):
    print(newlist[i])